In [1]:
import numpy as np
import PIL.Image as Im
import matplotlib.pyplot as plt
import math
import scipy as sp
import scipy.signal

In [ ]:
import matlab.engine
eng = matlab.engine.start_matlab()

# 1 Wiener filtering

### Question 1
$
r_n(x,0) = \sigma_n^2 * \delta(x,0) = /\text{formula collection, } \delta(x,0) = 1 / = \sigma_n^2 \implies
S_N(u,v) = \sigma_n^2
$


$
r_f(x,0) = \sigma_f^2 \cdot e^{\ln(\rho^{|x|})} = \sigma_f^2 \cdot e^{|x| \cdot \ln(\rho)} = \sigma_f^2 \cdot e^{-(-\ln\rho \cdot |x|)} = /\text{formula collection}/ = \frac{2(-\ln\rho)}{(-\ln\rho)^2 + (2\pi u)^2} \implies 
S_F(u,v) = \sigma_f^2 \cdot \frac{2(-ln\rho)}{(-ln\rho)^2 + (2\pi u)^2} 
$

Therefore:
\begin{align*}
S_F(u,v) &= \sigma_f^2 \cdot \frac{2(-ln\rho)}{(-ln\rho)^2 + (2\pi u)^2} \\
S_N(u,v) &= \sigma_n^2
\end{align*}

### Question 2
A inverse filter is defined as $\frac{1}{H(u,v)}$.

We have 
$
W(u,v)=\frac{H^{*}(u,v)}{|H(u,v)|^{2}+r\cdot S_{N}(u,v)/S_{F}(u,v)} = /r=0/ =
\frac{H^{*}(u,v)}{|H(u,v)|^{2}} =
\frac{H^{*}(u,v)}{H(u,v)\cdot H^{*}(u,v)} =
\frac{1}{H(u,v)}
$

In [ ]:
def mat_d(x):
    return matlab.double(x.tolist())

def wiener(f, h0, snr, rho, r):
    g = np.array(eng.circconv(matlab.double(f.tolist()), matlab.double(h0.tolist())))
    h = np.array(eng.addnoise(matlab.double(g.tolist()), matlab.double([snr])))
    return h, np.array(eng.wiener(mat_d(h), mat_d(h0), matlab.double([snr]), matlab.double([rho]), matlab.double([r])))


In [ ]:


f = np.array(Im.open("lab7files/skylt.tif"))
h0 = np.ones((5,5)) / 25.0
snr = 20
rho = 0.8
r = 1.0


h, fhat = wiener(f, h0, snr, rho, r)

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(131)
plt.imshow(f, cmap="gray")

plt.subplot(132)
plt.imshow(h, cmap="gray")

plt.subplot(133)
plt.imshow(fhat, cmap="gray")

### Question 3
Yes somewhat, the quality of the text is improved

In [ ]:
h1 = np.ones(1)
snr_1 = 10
snr_2 = 5
rho = 0.8
r = 1.0

h_1, fhat_1 = wiener(f, h1, snr_1, rho, r)
h_2, fhat_2 = wiener(f, h1, snr_2, rho, r)

plt.figure(figsize=(12,12))
plt.subplot(321)
plt.imshow(f, cmap="gray")

plt.subplot(323)
plt.imshow(h_1, cmap="gray")
plt.title("snr = 10")

plt.subplot(324)
plt.imshow(fhat_1, cmap="gray")

plt.subplot(325)
plt.imshow(h_2, cmap="gray")
plt.title("snr = 5")

plt.subplot(326)
plt.imshow(fhat_2, cmap="gray")

### Question 4
The text becomes more blurred, but the background noise is slightly smoothed out.

In [ ]:
h2 = np.ones((7,7))/49
h3 = np.ones((11,11))/121
snr = 40
rho = 0.8
r = 1.0

h_2, fhat_2 = wiener(f, h2, snr, rho, r)
h_3, fhat_3 = wiener(f, h3, snr, rho, r)

plt.figure(figsize=(12,12))
plt.subplot(321)
plt.imshow(f, cmap="gray")

plt.subplot(323)
plt.imshow(h_2, cmap="gray")
plt.title("h2 = ones(7,7)/49")

plt.subplot(324)
plt.imshow(fhat_2, cmap="gray")

plt.subplot(325)
plt.imshow(h_3, cmap="gray")
plt.title("h3 = ones(11,11)/121")

plt.subplot(326)
plt.imshow(fhat_3, cmap="gray")

### Question 5
The result is really good, you completely reconstruct the text. Background still a bit noisy.

In [ ]:
h3 = np.ones((11,11))/121
snr = 40
rho = 0.8
r_1 = 0.1
r_2 = 0


h_3_1, fhat_3_1 = wiener(f, h3, snr, rho, r_1)
h_3_2, fhat_3_2 = wiener(f, h3, snr, rho, r_2)

plt.figure(figsize=(12,12))
plt.subplot(321)
plt.imshow(f, cmap="gray")

plt.subplot(323)
plt.imshow(h_3_1, cmap="gray")
plt.title("h3 = ones(11,11)/121, r = 0.1")

plt.subplot(324)
plt.imshow(fhat_3_1, cmap="gray")

plt.subplot(325)
plt.imshow(h_3_2, cmap="gray")
plt.title("h3 = ones(11,11)/121, r = 0")

plt.subplot(326)
plt.imshow(fhat_3_2, cmap="gray")

### Question 6
* $r=0.1$ leads to good result
* $r=0$ leads to no guess of noise

In [ ]:
h3 = np.ones((11,11))/121
snr_1 = 15
snr_2 = 8
rho = 0.8
r_1 = 1


h_3_1, fhat_3_1 = wiener(f, h3, snr_1, rho, r_1)
h_3_2, fhat_3_2 = wiener(f, h3, snr_2, rho, r_1)

plt.figure(figsize=(12,12))
plt.subplot(321)
plt.imshow(f, cmap="gray")

plt.subplot(323)
plt.imshow(h_3_1, cmap="gray")
plt.title("h3 = ones(11,11)/121, r = 1, snr = 15")

plt.subplot(324)
plt.imshow(fhat_3_1, cmap="gray")

plt.subplot(325)
plt.imshow(h_3_2, cmap="gray")
plt.title("h3 = ones(11,11)/121, r = 1, snr = 8")

plt.subplot(326)
plt.imshow(fhat_3_2, cmap="gray")

### Question 7
* $SNR = 15$ leads to barely visible numbers
* $SNR = 8$ leads to barely visible text

In [ ]:
h3 = np.ones((11,11))/121
snr = 8
rho = 0.8
r_start = 0.1
r_end = 1


plt.figure(figsize=(12,60))
for r_i in range(10):
    h, fhat = wiener(f, h3, snr, rho, r_start + r_i/10)
    plt.subplot(10, 2, 2*r_i+1)
    plt.imshow(h, cmap="gray")
    plt.title("h3 = ones(11,11)/121, r = " + str(r_start + r_i/10) + ", snr = 15")
    plt.subplot(10, 2, 2*r_i+2)
    plt.imshow(fhat, cmap="gray")

### Question 8
It seems like the image quality becomes worse with higher r.
Even though that might be mathimatically correct.

The human might be able to interpret the image better when you do not account for noise. If you only try to fix the degraded image using inverse filtering, the brain might fix the noise on its own.

In [ ]:
h_blur = np.array(((0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1)))/15
print(h_blur)

### Question 9
See above
"Long filter" but not "tall". All values are at one side of the center.

In [ ]:
snr_1 = 40
snr_2 = 5

rho = 0.8
r_1 = 1.0
r_2 = 0.5

h_1_1, fhat_1_1 = wiener(f, h_blur, snr_1, rho, r_1)
h_2_1, fhat_2_1 = wiener(f, h_blur, snr_2, rho, r_1)
h_1_2, fhat_1_2 = wiener(f, h_blur, snr_1, rho, r_2)
h_2_2, fhat_2_2 = wiener(f, h_blur, snr_2, rho, r_2)

plt.figure(figsize=(16,40))
plt.subplot(521)
plt.imshow(f, cmap="gray")

plt.subplot(523)
plt.imshow(h_1_1, cmap="gray")
plt.title("snr = 40, r = 1")

plt.subplot(524)
plt.imshow(fhat_1_1, cmap="gray")

plt.subplot(525)
plt.imshow(h_2_1, cmap="gray")
plt.title("snr = 5, r = 1")


plt.subplot(526)
plt.imshow(fhat_2_1, cmap="gray")

plt.subplot(527)
plt.imshow(h_1_2, cmap="gray")
plt.title("snr = 40, r = 0.5")

plt.subplot(528)
plt.imshow(fhat_1_2, cmap="gray")

plt.subplot(529)
plt.imshow(h_2_2, cmap="gray")
plt.title("snr = 5, r = 0.5")

plt.subplot(5,2,10)
plt.imshow(fhat_2_2, cmap="gray")

The reconstructed filter looks really good. Almost like there is no degredation at all.
With low SNR the result is worse but the text is still readable.

### Question 10

## $\rho_x$:
$r_{f}(1,0) = \sigma_{f}^{2} \cdot \rho^{\sqrt{1^{2}+0^{2}}} = \sigma_{f}^{2} \cdot \rho^{1} = \sigma_{f}^{2} \cdot \rho$

$r_{f}(0,0) = \sigma_{f}^{2} \cdot \rho^{\sqrt{0^{2}+0^{2}}} = \sigma_{f}^{2} \cdot \rho^{0} = \sigma_{f}^{2} \cdot 1 = \sigma_{f}^{2}$

$\implies \rho_{x} = \frac{r_{f}(1,0)}{r_{f}(0,0)} = \frac{\sigma_{f}^{2} \cdot \rho}{\sigma_{f}^{2}} = \rho$

## $\rho_y$:
$r_{f}(0,1) = \sigma_{f}^{2} \cdot \rho^{\sqrt{0^{2}+1^{2}}} = \sigma_{f}^{2} \cdot \rho^{1} = \sigma_{f}^{2} \cdot \rho$

$\implies \rho_{y} = \frac{r_{f}(0,1)}{r_{f}(0,0)} = \frac{\sigma_{f}^{2} \cdot \rho}{\sigma_{f}^{2}} = \rho$

## $\rho$:
$\rho = \frac{\rho_{x} + \rho_{y}}{2} = \frac{\rho + \rho}{2} = \frac{2\rho}{2} = \rho$

In [ ]:
def autocorr(im):
    f_hat = np.mean(im)
    f_im = np.fft.ifftshift(np.fft.fft2(np.fft.fftshift(im-f_hat)))
    f = f_im.conj() * f_im
    return np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(f)))

def rho(f):
    return (f[1,0]/f[0,0]+f[0,1]/f[0,0])/2

In [ ]:
acorr_s = autocorr(f)
corr_s = (acorr_s.real+acorr_s.imag).astype(np.int64)
plt.figure(figsize=(16,8))
plt.subplot(131)
plt.imshow(corr_s, cmap="gray")
plt.title("autocorrelation of skylt")

foppa = np.array(Im.open("lab7files/foppa.tif"))
acorr_f = autocorr(foppa)
corr_f = (acorr_f.real+acorr_f.imag).astype(np.int64)
plt.subplot(132)
plt.imshow(corr_f, cmap="gray")
plt.title("autocorrelation of foppa")

baboon = np.array(Im.open("lab7files/baboon256.tif"))
acorr_b = autocorr(baboon)
corr_b = (acorr_b.real+acorr_b.imag).astype(np.int64)
plt.subplot(133)
plt.imshow(corr_b, cmap="gray")
plt.title("autocorrelation of baboon")

plt.figure(figsize=(16,8))
plt.subplot(131)
plt.imshow(f, cmap="gray")

plt.subplot(132)
plt.imshow(foppa, cmap="gray")

plt.subplot(133)
plt.imshow(baboon, cmap="gray")

plt.show()

### Question 11
At least Foppa looks similar to the sketch. The others look like the sketches at least in the middle.

One reason for bad orders can be because of convolution outside of bounds.

### Question 12
See below

In [ ]:
print(f"rho for the image skylt is: {rho(np.fft.fftshift(corr_s))}")
print(f"rho for the image foppa is: {rho(np.fft.fftshift(corr_f))}")
print(f"rho for the image baboon is: {rho(np.fft.fftshift(corr_b))}")


# 2 Edge detection with Canny

In [ ]:
hus = np.array(Im.open("lab7files/hus.tif"))

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(9,9))

sobelx = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])/8;
Imsobelx = eng.conv2(hus, sobelx, 'same');
sobely = sobelx.T;
Imsobely = eng.conv2(hus, sobely, 'same');
Imgrad = np.sqrt(np.power(Imsobelx,2)+np.power(Imsobely,2));
maxv = np.max(Imgrad);
binvect = np.arange(0,maxv, maxv//100);
histo = np.histogram(Imgrad, binvect);

T=22;
ImgradT = Imgrad>T;

ax1.imshow(hus, cmap='grey')
ax1.set_title("original")

ax2.imshow(Imgrad, cmap='grey')
ax2.set_title("magngrad")

ax3.plot(histo[0])
ax3.set_title("histogram")

ax4.imshow(ImgradT, cmap='grey')
ax4.set_title("Tresholded")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

Imskel = eng.bwmorph(ImgradT, 'skel', np.inf);

ax.imshow(Imskel, cmap='grey')
ax.set_title(f"Skeleton with T={T}")
plt.show()

### Question 13
No, too low threshold ~22 removes the chimney's right part but keeps some part sticking up while too high threshold keeps to much of the tiles.
Even with T=22 we still have many tiles

### Question 14
We chose T=22
Keeps part of chimney while minimizing tiles

## Canny
### Question 15
By "traversing" the edge we get more precise max values instead of also getting other values by thresholding.
By looking at the edge normal, we introduce another requirement and therefore select less values.

### Question 16
The hysteresis is important in the canny algorithm as it make sure that your edge segments are continuous, since it reduces the amount of unconnected edges.
Another benefit is that is helps filtering out potential false positives by only allowing the pixels that are between the low and high threshold to be treated as part of an edge if they are adjacent to pixels that have been marked as strong edges (above high threshold or (recursively) neighboring pixels)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

[cannyim, T_aut] = eng.edge(hus, "canny", nargout=2);

ax.imshow(cannyim, cmap='grey')
ax.set_title("Canny")
plt.show()

In [ ]:
print(f"Threshold 1: {T_aut[0][0]}, Threshold 2: {T_aut[0][1]}")

### Question 17
As seen above, it selects
0.01875 and
0.046875

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

T = 0.25
[cannyim2, T_aut] = eng.edge(hus, "canny", matlab.double([0.4*T, T]), nargout=2);

ax.imshow(cannyim2) 
ax.set_title(f"Canny with T={T}")
plt.show()

### Question 18
We select T=0.25
see matlab image

In [ ]:
fig, ax = plt.subplots(figsize=(1,1))

T = 0.25
[cannyim2, T_aut] = eng.edge(hus, "canny", matlab.double([0.4*T, T]), nargout=2);

ax.imshow(cannyim2)
ax.set_title(f"Canny with T={T}, smaller")
plt.show()

### Question 19
This is since the width of the edge lines are 1 pixel, meaning when we scale down the image below it's original resolution, we will have missing data due to downscaling algorithms

This issue is actually not really present in the python version, probably because they are using a different algorithm for scaling images, here is a screenshot from the same image in matlab scaled down:

![Question 19 image](images/tsbb19.png "Question 19")

# 3 The Hough transform

### Question 20
![Question 20 sketch](images/tsbb20.png "Question 20")
We manually computed the position (theta, rho) in the hough plot at the theta=-90, -45, 0, 45 and 90 positions. This gave us the parabola (actually sinusoidal) line.

### Question 21
![Question 21 sketch](images/tsbb21.png "Question 21")

Since we only have one point, this will draw each possible line found in the hough transform. Each line is "equally possible" through one singular point.

### DEMO E

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12,12))

im = np.zeros((64,64)); im[0,0] = 1;
im[15,15] = 1;
im[31,31] = 1;
im[47,47] = 1;
im[63,63] = 1;
ax1.imshow(im, origin='lower')
ax1.set_title("original")

[H, T, R] = eng.hough(im, 'Theta', matlab.double(np.arange(-90,90).tolist()), nargout=3)
ax2.imshow(H, extent=[-90, 90, 90, -90])
ax2.set_title("Hough transform")


P = eng.houghpeaks(H, matlab.double(10),'threshold', np.ceil(0.5*np.max(H)))
x = T[0][int(P[0][1])-1]
y = R[0][int(P[0][0])-1]
ax2.plot(x, y, marker='s', linestyle='-', markerfacecolor='none', color='red')

ax3.imshow(im, origin='lower')
lines = eng.houghlines(np.ones((64, 64)),T,R,P)
xp = (lines['point1'][0][0]-1, lines['point2'][0][0]-1)
yp = (lines['point1'][0][1]-1, lines['point2'][0][1]-1)
ax3.plot(xp, yp, linestyle='-', color='lime')
ax3.set_title("Hough Line")
plt.show()

### Question 22
Since all dots are in a straight line, in the Hough-curve, we get exactly one point where the lines for each dot intersects, which corresponds to the line that crosses all dots

### DEMO F

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12,12))

im = np.array(Im.open("lab7files/polyg.tif"))
ax1.imshow(im, origin='lower')
ax1.set_title("Original")

#Canny
T = 0.45
[cannyim, T_aut] = eng.edge(im, "canny", matlab.double([0.4*T, T]), nargout=2);
ax2.imshow(cannyim, origin='lower')
ax2.set_title("Canny")

[H, T, R] = eng.hough(cannyim, 'Theta', matlab.double(np.arange(-90,90).tolist()), nargout=3)
ax3.imshow(H, extent=[-90, 90, 361, -361], aspect='auto')
ax3.set_title("Hough transform")


P = eng.houghpeaks(H, matlab.double(4),'threshold', np.ceil(0.5*np.max(H)))
for peak in P:
    x = T[0][int(peak[1])-1]
    y = R[0][int(peak[0])-1]
    ax3.plot(x, y, marker='s', linestyle='-', markerfacecolor='none', color='red')

ax4.imshow(im, origin='lower')
# pythons matlabengine cannot return the value of houghlines if you have more than one line, so we ran it in
# matlab and pasted the xy values
#   point1: [165 1]
#   point2: [210 256]
#    theta: -10
#      rho: 162

#   point1: [1 168]
#   point2: [256 208]
#    theta: -81
#      rho: -165

#   point1: [88 1]
#   point2: [24 256]
#    theta: 14
#      rho: 84

#   point1: [1 69]
#   point2: [256 20]
#    theta: 79
#      rho: 67
#lines = eng.houghlines(np.ones((256, 256)),T,R,P)
lines = [[[165, 209], [0, 255]], [[0, 255], [167, 207]], [[87, 23], [0,255]], [[0, 255], [68, 19]]]
for line in lines:
    xp = line[0]
    yp = line[1]
    ax4.plot(xp, yp, linestyle='-', color='lime')
ax4.set_title("Hough lines")
plt.show()

### DEMO G

In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(12,9))

im = np.array(Im.open("lab7files/polyg.tif"))
ax1.imshow(im, origin='lower')
ax1.set_title("Original")

#Canny
T = 0.45
[cannyim, T_aut] = eng.edge(im, "canny", matlab.double([0.4*T, T]), nargout=2);
ax2.imshow(cannyim, origin='lower')
ax2.set_title("Canny")

se = eng.strel('disk', matlab.double(2));
into_hough = eng.imdilate(cannyim,se)
ax3.imshow(into_hough, origin='lower')
ax3.set_title("Dilated canny")

[H, T, R] = eng.hough(cannyim, 'Theta', matlab.double(np.arange(-90,90).tolist()), nargout=3)
ax4.imshow(H, extent=[-90, 90, 361, -361], aspect='auto')
ax4.set_title("Hough transform")


P = eng.houghpeaks(H, matlab.double(4),'threshold', np.ceil(0.5*np.max(H)))
for peak in P:
    x = T[0][int(peak[1])-1]
    y = R[0][int(peak[0])-1]
    ax4.plot(x, y, marker='s', linestyle='-', markerfacecolor='none', color='red')


# pythons matlabengine cannot return the value of houghlines if you have more than one line, so we ran it in
# matlab and pasted the xy values
#   point1: [171 35]
#   point2: [200 199]
#    theta: -10
#      rho: 162

#   point1: [44 175]
#   point2: [199 199]
#    theta: -81
#      rho: -165

#   point1: [75 53]
#   point2: [44 175]
#    theta: 14
#      rho: 84

#   point1: [73 55]
#   point2: [173 36]
#    theta: 79
#      rho: 67
# lines = houghlines(into_hough,T,R,P,"FillGap",40,"MinLength",40);
ax5.imshow(im, origin='lower')
lines = [[[171, 199], [35, 198]], [[43, 198], [174, 198]], [[74, 43], [52, 174]], [[72, 172], [54, 35]]]
for line in lines:
    xp = line[0]
    yp = line[1]
    ax5.plot(xp, yp, linestyle='-', color='lime')
ax5.set_title("Hough lines")
plt.show()